<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Project-1:-Airline-Flight" data-toc-modified-id="Project-1:-Airline-Flight-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Project 1: Airline Flight</a></span></li><li><span><a href="#Data-cleaning:" data-toc-modified-id="Data-cleaning:-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data cleaning:</a></span><ul class="toc-item"><li><span><a href="#Indexing:-Categorical-data-to-numerical" data-toc-modified-id="Indexing:-Categorical-data-to-numerical-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Indexing: Categorical data to numerical</a></span></li><li><span><a href="#Assembling-columns" data-toc-modified-id="Assembling-columns-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Assembling columns</a></span></li></ul></li><li><span><a href="#Classification:-Decision-Tree" data-toc-modified-id="Classification:-Decision-Tree-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Classification: Decision Tree</a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression-model" data-toc-modified-id="Logistic-Regression-model-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Logistic Regression model</a></span></li><li><span><a href="#Evaluate-the-Logistic-Regression-model" data-toc-modified-id="Evaluate-the-Logistic-Regression-model-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Evaluate the Logistic Regression model</a></span></li></ul></li><li><span><a href="#Regression:-Predicting-flight-delay-by-the-distance-and-airport:" data-toc-modified-id="Regression:-Predicting-flight-delay-by-the-distance-and-airport:-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Regression: Predicting flight delay by the distance and airport:</a></span><ul class="toc-item"><li><span><a href="#Encoding-flight-origin" data-toc-modified-id="Encoding-flight-origin-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Encoding flight origin</a></span></li></ul></li><li><span><a href="#Feature-engineering:--Adding-departure-time" data-toc-modified-id="Feature-engineering:--Adding-departure-time-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Feature engineering:  Adding departure time</a></span><ul class="toc-item"><li><span><a href="#Bucketing" data-toc-modified-id="Bucketing-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Bucketing</a></span></li><li><span><a href="#Redo-Regression" data-toc-modified-id="Redo-Regression-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Redo Regression</a></span></li></ul></li><li><span><a href="#Regularization:-Feature-selection" data-toc-modified-id="Regularization:-Feature-selection-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Regularization: Feature selection</a></span><ul class="toc-item"><li><span><a href="#Implementration-to-our-data" data-toc-modified-id="Implementration-to-our-data-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Implementration to our data</a></span></li></ul></li><li><span><a href="#SMS-spam-data" data-toc-modified-id="SMS-spam-data-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>SMS spam data</a></span><ul class="toc-item"><li><span><a href="#Data-Preprocessing:" data-toc-modified-id="Data-Preprocessing:-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Data Preprocessing:</a></span></li><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Logistic Regression</a></span></li></ul></li></ul></div>

Spark is a platform for cluster computing. Spark lets you spread data and computations over clusters with multiple nodes (think of each node as a separate computer).It is much faster that hadoop. Splitting up your data makes it easier to work with very large datasets because each node only works with a small amount of data.

The SparkSession class has a builder attribute, which is an instance of the Builder class. The Builder class exposes three important methods that let you:

- specify the location of the master node;
- name the application (optional); and
- retrieve an existing SparkSession or, if there is none, create a new one.
The SparkSession class has a version attribute which gives the version of Spark.

In [12]:
import findspark
findspark.init('C:/Users/Hamid/Anaconda3/lib/site-packages/pyspark')
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession

In [13]:
# Create SparkSession object
spark = SparkSession.builder.master('local[*]').appName('test').getOrCreate()

# What version of Spark?
print(spark.version)

# Terminate the cluster
#spark.stop()

#SparkSession._instantiatedContext = None

2.4.4


#  Project 1: Airline Flight
We will develope a model which predice whether or not a given flight will be delayed.  

read.csv methode read all the data as a string as default. To make it correct we need to use  inferSchema=True. However, It need another checking of the data which can take so much time if data is big. 

#  Data cleaning:

- removing an uninformative column 
- removing rows which do not have information about whether or not a flight was delayed.

In [14]:
# Read data from CSV file, inferSchema - deduce column data types from data.
flights = spark.read.csv('flights.csv',
                         sep=',',
                         header=True,
                         inferSchema=True,
                         nullValue='NA')

# Get number of records
print("The data contain %d records." % flights.count())

# View the first five records
flights.show(5)

# Check column data types
flights.dtypes

The data contain 50000 records.
+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351| null|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|
|  4|  2|  5|     AA|   325|ORD| 258|  8.92|      65| null|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows



[('mon', 'int'),
 ('dom', 'int'),
 ('dow', 'int'),
 ('carrier', 'string'),
 ('flight', 'int'),
 ('org', 'string'),
 ('mile', 'int'),
 ('depart', 'double'),
 ('duration', 'int'),
 ('delay', 'int')]

In [15]:
# Remove the 'flight' column
flights_drop_column = flights.drop('flight')

# Number of records with missing 'delay' values. delay IS NULL is sql syntax
flights_drop_column.filter('delay IS NULL').count()

# Remove records with missing 'delay' values
flights = flights_drop_column.filter('delay IS NOT NULL')

# Remove records with missing values in any column and get the number of remaining rows
flights_none_missing = flights.dropna()
print("Intial 50000 data contain %d not null records." % flights_none_missing.count())


Intial 50000 data contain 47022 not null records.


The Federal Aviation Administration (FAA) considers a flight to be "delayed" when it arrives 15 minutes or more after its scheduled time.

The next step of preparing the flight data has two parts:
- the units of distance, replacing the mile column with a km column; and
- create a Boolean column indicating whether or not a flight was delayed.





Spark withColumn() function is used to rename, change the value, convert the datatype of an existing DataFrame column and also can be used to create a new column, on this post, I will walk you through commonly used DataFrame column operations with Scala and Pyspark examples.



In [16]:
# Import the required function
from pyspark.sql.functions import round

# Convert 'mile' to 'km' and drop 'mile' column
flights_km = flights.withColumn('km', round(flights.mile * 1.60934, 0)) \
                    .drop('mile')

# Create 'label' column indicating whether flight delayed (1) or not (0)
# with cast we change the datatype to integer
flights = flights_km.withColumn('label', (flights_km.delay >= 15).cast('integer'))

# Check first five records
flights.show(5)

+---+---+---+-------+---+------+--------+-----+------+-----+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|label|
+---+---+---+-------+---+------+--------+-----+------+-----+
|  0| 22|  2|     UA|ORD| 16.33|      82|   30| 509.0|    1|
|  2| 20|  4|     UA|SFO|  6.17|      82|   -8| 542.0|    0|
|  9| 13|  1|     AA|ORD| 10.33|     195|   -5|1989.0|    0|
|  5|  2|  1|     UA|SFO|  7.98|     102|    2| 885.0|    0|
|  7|  2|  6|     AA|ORD| 10.83|     135|   54|1180.0|    1|
+---+---+---+-------+---+------+--------+-----+------+-----+
only showing top 5 rows



## Indexing: Categorical data to numerical
In the flights data there are two columns, carrier and org, which hold categorical data. We need to transform those columns into indexed numerical values.

In [17]:
from pyspark.ml.feature import StringIndexer

# Create an indexer
indexer = StringIndexer(inputCol='carrier', outputCol='carrier_idx')

# Indexer identifies categories in the data
indexer_model = indexer.fit(flights)

# Indexer creates a new column with numeric index values
flights_indexed = indexer_model.transform(flights)

# Repeat the process for the other categorical feature
flights_indexed = StringIndexer(inputCol='org', outputCol='org_idx').fit(flights_indexed).transform(flights_indexed)

flights_indexed.show(5)

+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|label|carrier_idx|org_idx|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+
|  0| 22|  2|     UA|ORD| 16.33|      82|   30| 509.0|    1|        0.0|    0.0|
|  2| 20|  4|     UA|SFO|  6.17|      82|   -8| 542.0|    0|        0.0|    1.0|
|  9| 13|  1|     AA|ORD| 10.33|     195|   -5|1989.0|    0|        1.0|    0.0|
|  5|  2|  1|     UA|SFO|  7.98|     102|    2| 885.0|    0|        0.0|    1.0|
|  7|  2|  6|     AA|ORD| 10.83|     135|   54|1180.0|    1|        1.0|    0.0|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+
only showing top 5 rows



## Assembling columns
The final stage of data preparation is to consolidate all of the predictor columns into a single column.

In [18]:
# Import the necessary class
from pyspark.ml.feature import VectorAssembler

# Create an assembler object
assembler = VectorAssembler(inputCols=[
    'mon', 'dom', 'dow', 'carrier_idx', 'org_idx', 'km', 'depart', 'duration'
], outputCol='features')

# Consolidate predictor columns
flights_assembled = assembler.transform(flights_indexed)

# Check the resulting column
flights_assembled.select('features', 'delay').show(5, truncate=False)

+-----------------------------------------+-----+
|features                                 |delay|
+-----------------------------------------+-----+
|[0.0,22.0,2.0,0.0,0.0,509.0,16.33,82.0]  |30   |
|[2.0,20.0,4.0,0.0,1.0,542.0,6.17,82.0]   |-8   |
|[9.0,13.0,1.0,1.0,0.0,1989.0,10.33,195.0]|-5   |
|[5.0,2.0,1.0,0.0,1.0,885.0,7.98,102.0]   |2    |
|[7.0,2.0,6.0,1.0,0.0,1180.0,10.83,135.0] |54   |
+-----------------------------------------+-----+
only showing top 5 rows



# Classification: Decision Tree
1. Splilt data into train and test
2. improt object
3. Prediction and test on the test set. 
4. Creat confusion matrix



A confusion matrix gives a useful breakdown of predictions versus known values. It has four cells which represent the counts of:

- True Negatives (TN) — model predicts negative outcome & known outcome is negative
- True Positives (TP) — model predicts positive outcome & known outcome is positive
- False Negatives (FN) — model predicts negative outcome but known outcome is positive
- False Positives (FP) — model predicts positive outcome but known outcome is negative.

In [27]:
from pyspark.ml.classification import DecisionTreeClassifier

# Split into training and testing sets in a 80:20 ratio
flights_train, flights_test = flights_assembled.randomSplit([0.8, 0.2], seed=17)

# Check that training set has around 80% of records
training_ratio = flights_train.count() / flights.count()
print('train to test ratio=',training_ratio)


# Create a classifier object and fit to the training data
tree = DecisionTreeClassifier()
tree_model = tree.fit(flights_train)

print("Probability of predictions:")
# Create predictions for the testing data and take a look at the predictions
prediction = tree_model.transform(flights_test)


prediction.select('label', 'prediction', 'probability').show(5, False)

# Create a confusion matrix
print("Confusion matrix:")
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()

# Accuracy measures the proportion of correct predictions
accuracy = (TN + TP) / (TN + TP + FN + FP)
print('accuracy=', accuracy)

train to test ratio= 0.7980732423121092
Probability of predictions:
+-----+----------+---------------------------------------+
|label|prediction|probability                            |
+-----+----------+---------------------------------------+
|1    |1.0       |[0.4939987365761213,0.5060012634238787]|
|1    |1.0       |[0.3550259700580507,0.6449740299419493]|
|1    |1.0       |[0.3550259700580507,0.6449740299419493]|
|1    |1.0       |[0.3550259700580507,0.6449740299419493]|
|1    |1.0       |[0.3550259700580507,0.6449740299419493]|
+-----+----------+---------------------------------------+
only showing top 5 rows

Confusion matrix:
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 1205|
|    0|       0.0| 2400|
|    1|       1.0| 3619|
|    0|       1.0| 2271|
+-----+----------+-----+

accuracy= 0.6339125855713533


## Logistic Regression model
The objective is to predict whether a flight is likely to be delayed by at least 15 minutes (label 1) or not (label 0).

## Evaluate the Logistic Regression model

Accuracy is generally not a very reliable metric because it can be biased by the most common target class
Precision is the proportion of positive predictions which are correct. For all flights which are predicted to be delayed, what proportion is actually delayed?

Recall is the proportion of positives outcomes which are correctly predicted. For all delayed flights, what proportion is correctly predicted by the model?

The precision and recall are generally formulated in terms of the positive target class. But it's also possible to calculate weighted versions of these metrics which look at both target classes.

The components of the confusion matrix are available as TN, TP, FN and FP, as well as the object prediction


ROC and AUC : The ROC curve plots the true positive rate versus the false positve for different threshold values.
AUS summersizes the ROC in single number which is the area under ROC curve.

A prefect model which act prefectly regardless to treshhold value will have AUC of 1.

In [28]:
# Import the logistic regression class
from pyspark.ml.classification import LogisticRegression

# Create a classifier object and train on training data
logistic = LogisticRegression().fit(flights_train)

# Create predictions for the testing data and show confusion matrix
prediction = logistic.transform(flights_test)
prediction.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 1652|
|    0|       0.0| 2645|
|    1|       1.0| 3172|
|    0|       1.0| 2026|
+-----+----------+-----+



In [33]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

# Calculate precision and recall
precision = TP / (TP + FP)
recall = TP / (TP + FN)
print('precision = {:.2f}\nrecall    = {:.2f}'.format(precision, recall))

# Find weighted precision
multi_evaluator = MulticlassClassificationEvaluator()
weighted_precision = multi_evaluator.evaluate(prediction, {multi_evaluator.metricName: "weightedPrecision"})

# Find AUC
binary_evaluator = BinaryClassificationEvaluator()
auc = binary_evaluator.evaluate(prediction, {binary_evaluator.metricName: "areaUnderROC"})
print('auc = {:.2f}'.format(auc))


precision = 0.61
recall    = 0.75
auc = 0.65


# Regression: Predicting flight delay by the distance and airport:
We want to see the distance and  type of airport can effect the delay or not. Bigger airport might take more time for checkin.

So in assembler we just use column 'km' and origin airport (which is encoded)  as features and duration is the prediction. 


Interpreting the coefficients
The linear regression model for flight duration as a function of distance takes the form

duration=α+β×distance+β1×airport
where

α — intercept (component of duration which does not depend on distance) and
β — coefficient (rate at which duration increases as a function of distance; also called the slope).

## Encoding flight origin
The org column in the flights data is a categorical variable giving the airport from which a flight departs.

ORD — O'Hare International Airport (Chicago)

SFO — San Francisco International Airport

JFK — John F Kennedy International Airport (New York)

LGA — La Guardia Airport (New York)

SMF — Sacramento

SJC — San Jose

TUS — Tucson International Airport

OGG — Kahului (Hawaii)


For this part we used the 

In [43]:
# Import the one hot encoder class
from pyspark.ml.feature import OneHotEncoderEstimator

# Create an instance of the one hot encoder
onehot = OneHotEncoderEstimator(inputCols=['org_idx'], outputCols=['org_dummy'])

# Apply the one hot encoder to the flights data
onehot = onehot.fit(flights_indexed)
flights_onehot = onehot.transform(flights_indexed)

# Check the results
flights_onehot.select('org', 'org_idx', 'org_dummy').distinct().sort('org_idx').show()



+---+-------+-------------+
|org|org_idx|    org_dummy|
+---+-------+-------------+
|ORD|    0.0|(7,[0],[1.0])|
|SFO|    1.0|(7,[1],[1.0])|
|JFK|    2.0|(7,[2],[1.0])|
|LGA|    3.0|(7,[3],[1.0])|
|SMF|    4.0|(7,[4],[1.0])|
|SJC|    5.0|(7,[5],[1.0])|
|TUS|    6.0|(7,[6],[1.0])|
|OGG|    7.0|    (7,[],[])|
+---+-------+-------------+



In [44]:
# Import the necessary class
from pyspark.ml.feature import VectorAssembler

# Create an assembler object
assembler = VectorAssembler(inputCols=['km', 'org_dummy'], outputCol='features')

# Consolidate predictor columns
flights_assembled = assembler.transform(flights_onehot)


In [47]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Check the resulting column
flights_assembled.select('features', 'delay').show(5, truncate=False)


# Split into training and testing sets in a 80:20 ratio
flights_train, flights_test = flights_assembled.randomSplit([0.8, 0.2], seed=17)

# Create a regression object and train on training data
regression = LinearRegression(labelCol='duration').fit(flights_train)

# Create predictions for the testing data and take a look at the predictions
predictions = regression.transform(flights_test)
predictions.select('duration', 'prediction').show(5, False)

# Calculate the RMSE
RegressionEvaluator(labelCol='duration').evaluate(predictions)

# Average speed in km per hour
avg_speed_hour = 60 / regression.coefficients[0]
print(" Average speed is %d km per houre" % avg_speed_hour)


# Intercept (average minutes on ground)
inter = regression.intercept
print('Intercept',inter)

# Average minutes on ground at JFK
avg_ground_jfk = inter + regression.coefficients[3]
print('Average minutes on ground at JFK=',avg_ground_jfk)

# Average minutes on ground at LGA
avg_ground_lga = inter + regression.coefficients[4]
print('Average minutes on ground at LGA=',avg_ground_lga)

+----------------------+-----+
|features              |delay|
+----------------------+-----+
|(8,[0,1],[509.0,1.0]) |30   |
|(8,[0,2],[542.0,1.0]) |-8   |
|(8,[0,1],[1989.0,1.0])|-5   |
|(8,[0,2],[885.0,1.0]) |2    |
|(8,[0,1],[1180.0,1.0])|54   |
+----------------------+-----+
only showing top 5 rows

+--------+------------------+
|duration|prediction        |
+--------+------------------+
|240     |228.3383341007048 |
|160     |150.56907837678995|
|130     |132.0788580796877 |
|275     |265.0747799757123 |
|85      |92.64790636177084 |
+--------+------------------+
only showing top 5 rows

 Average speed is 807 km per houre
Intercept 15.536278501461982
Average minutes on ground at JFK= 68.2929992642743
Average minutes on ground at LGA= 62.37189890382314


# Feature engineering:  Adding departure time
## Bucketing
Here we work on manipulating features. we assign a lable for each range of an specific feature. The resulting categorical variable is often more powerfull. 
Then we make them one-hot encoded. 
The main  goal is to produce new features which has more effect on the target.
In our dataset we convert the flight departure times from numeric values between 0 (corresponding to 00:00) and 24 (corresponding to 24:00) to binned values. You'll then take those binned values and one-hot encode them.


The data are in flights. The km, org_dummy and depart_dummy columns have been assembled into features, where km is index 0, org_dummy runs from index 1 to 7 and depart_dummy from index 8 to 14.


In [49]:
from pyspark.ml.feature import Bucketizer, OneHotEncoderEstimator

# Create buckets at 3 hour intervals through the day
buckets = Bucketizer(splits=[0, 3, 6, 9, 12, 15, 18, 21, 24], inputCol='depart', outputCol='depart_bucket')

# Bucket the departure times
bucketed = buckets.transform(flights_onehot)
bucketed.select('depart', 'depart_bucket').show(5)

# Create a one-hot encoder for depart_bucket
onehot = OneHotEncoderEstimator(inputCols=['depart_bucket'], outputCols=['depart_dummy'])

# One-hot encode the bucketed departure times
flights_onehot_bucket = onehot.fit(bucketed).transform(bucketed)
flights_onehot_bucket.select('depart', 'depart_bucket', 'depart_dummy').show(5)


+------+-------------+
|depart|depart_bucket|
+------+-------------+
| 16.33|          5.0|
|  6.17|          2.0|
| 10.33|          3.0|
|  7.98|          2.0|
| 10.83|          3.0|
+------+-------------+
only showing top 5 rows

+------+-------------+-------------+
|depart|depart_bucket| depart_dummy|
+------+-------------+-------------+
| 16.33|          5.0|(7,[5],[1.0])|
|  6.17|          2.0|(7,[2],[1.0])|
| 10.33|          3.0|(7,[3],[1.0])|
|  7.98|          2.0|(7,[2],[1.0])|
| 10.83|          3.0|(7,[3],[1.0])|
+------+-------------+-------------+
only showing top 5 rows



## Redo Regression

In [50]:
# Import the necessary class
from pyspark.ml.feature import VectorAssembler

# Create an assembler object
assembler = VectorAssembler(inputCols=[
     'km', 'org_dummy', 'depart_dummy'
], outputCol='features')

# Consolidate predictor columns
flights_assembled_kod = assembler.transform(flights_onehot_bucket)


In [51]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Check the resulting column
flights_assembled_kod.select('features', 'delay').show(5, truncate=False)
flights_assembled_kod.show(5)

# Split into training and testing sets in a 80:20 ratio
flights_train, flights_test = flights_assembled_kod.randomSplit([0.8, 0.2], seed=17)

# Create a regression object and train on training data
regression = LinearRegression(labelCol='duration').fit(flights_train)

# Create predictions for the testing data and take a look at the predictions
predictions = regression.transform(flights_test)
predictions.select('duration', 'prediction').show(5, False)

# Calculate the RMSE
RegressionEvaluator(labelCol='duration').evaluate(predictions)

# Average minutes on ground at OGG for flights departing between 21:00 and 24:00
avg_eve_ogg = regression.intercept
print(avg_eve_ogg)

# Average minutes on ground at OGG for flights departing between 00:00 and 03:00
avg_night_ogg = regression.intercept + regression.coefficients[8]
print(avg_night_ogg)

# Average minutes on ground at JFK for flights departing between 00:00 and 03:00
avg_night_jfk = regression.intercept + regression.coefficients[8] + regression.coefficients[3]
print(avg_night_jfk)

+------------------------------+-----+
|features                      |delay|
+------------------------------+-----+
|(15,[0,1,13],[509.0,1.0,1.0]) |30   |
|(15,[0,2,10],[542.0,1.0,1.0]) |-8   |
|(15,[0,1,11],[1989.0,1.0,1.0])|-5   |
|(15,[0,2,10],[885.0,1.0,1.0]) |2    |
|(15,[0,1,11],[1180.0,1.0,1.0])|54   |
+------------------------------+-----+
only showing top 5 rows

+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+-------------+-------------+-------------+--------------------+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|label|carrier_idx|org_idx|    org_dummy|depart_bucket| depart_dummy|            features|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+-------------+-------------+-------------+--------------------+
|  0| 22|  2|     UA|ORD| 16.33|      82|   30| 509.0|    1|        0.0|    0.0|(7,[0],[1.0])|          5.0|(7,[5],[1.0])|(15,[0,1,13],[509...|
|  2| 20|  4|     UA|SFO|  6.17|      82|   -8| 

# Regularization: Feature selection
is usefull for feature selection when we have many column. An extra regularization term is added to the loss function.
The regularization term can be either
- Lasso — absolute value ofthe coefcients
- Ridge — square ofthe coefcients

It's also possible to have a blend of Lasso and Ridge regression.

Strength of regularization determined by parameter λ:

λ = 0 — no regularization (standard regression) 

λ = ∞ — complete regularization (all coefcients zero)

In lasso all values can be close to zero but it Ridge they could be zero.
Elastic net parameter is a hyperpatameter which should be zero.


## Implementration to our data
More features will always make the model more complicated and difficult to interpret.

These are the features you'll include in the next model:

- km
- org (origin airport, one-hot encoded, 8 levels)
- depart (departure time, binned in 3 hour intervals, one-hot encoded, 8 levels)
- dow (departure day of week, one-hot encoded, 7 levels) and
- mon (departure month, one-hot encoded, 12 levels).
- These have been assembled into the features column, which is a sparse representation of 32 columns (remember one-hot encoding produces a number of columns which is one fewer than the number of levels).

In [52]:

# Import the one hot encoder class
from pyspark.ml.feature import OneHotEncoderEstimator

# Create an instance of the one hot encoder
onehot = OneHotEncoderEstimator(inputCols=['dow'], outputCols=['dow_dummy'])

# Apply the one hot encoder to the flights data
onehot = onehot.fit(flights_assembled_kod)
flights_onehot_a = onehot.transform(flights_assembled_kod)

# Create an instance of the one hot encoder
onehot = OneHotEncoderEstimator(inputCols=['mon'], outputCols=['mon_dummy'])

# Apply the one hot encoder to the flights data
onehot = onehot.fit(flights_onehot_a)
flights_onehot_all = onehot.transform(flights_onehot_a)
flights_onehot_all= flights_onehot_all.drop('features')
# Check the results
flights_onehot_all.select( 'km','depart_dummy', 'org_dummy', 'dow_dummy', 'mon_dummy').distinct().show(5)


+------+-------------+-------------+-------------+--------------+
|    km| depart_dummy|    org_dummy|    dow_dummy|     mon_dummy|
+------+-------------+-------------+-------------+--------------+
|3954.0|(7,[5],[1.0])|(7,[1],[1.0])|    (6,[],[])|(11,[0],[1.0])|
|1754.0|(7,[1],[1.0])|(7,[2],[1.0])|(6,[2],[1.0])|(11,[9],[1.0])|
| 301.0|(7,[2],[1.0])|(7,[2],[1.0])|(6,[5],[1.0])|(11,[6],[1.0])|
| 542.0|(7,[2],[1.0])|(7,[1],[1.0])|(6,[2],[1.0])|(11,[9],[1.0])|
| 425.0|    (7,[],[])|(7,[0],[1.0])|(6,[5],[1.0])|(11,[4],[1.0])|
+------+-------------+-------------+-------------+--------------+
only showing top 5 rows



In [53]:
# Import the necessary class
from pyspark.ml.feature import VectorAssembler

# Create an assembler object
assembler = VectorAssembler(inputCols=[
     'km', 'org_dummy', 'depart_dummy', 'dow_dummy', 'mon_dummy'
], outputCol='features')

# Consolidate predictor columns
flights_assembled_kod_5f = assembler.transform(flights_onehot_all)

In [54]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Split into training and testing sets in a 80:20 ratio
flights_train, flights_test = flights_assembled_kod_5f.randomSplit([0.8, 0.2], seed=17)

# Fit Lasso model (α = 1) to training data
regression = LinearRegression(labelCol='duration', regParam=1, elasticNetParam=1).fit(flights_train)

# Calculate the RMSE on testing data
rmse = RegressionEvaluator(labelCol='duration').evaluate(regression.transform(flights_test))
print("The test RMSE is", rmse)

# Look at the model coefficients
coeffs = regression.coefficients
print(coeffs)

# Number of zero coefficients
zero_coeff = sum([beta == 0 for beta in regression.coefficients])
print("Number of coefficients equal to 0:", zero_coeff)

The test RMSE is 11.65852539759888
[0.07347781392862278,5.705478011506563,0.0,28.958072294272437,21.75674278399281,-2.342239176353107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0653074231639355,1.2051514011784286,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
Number of coefficients equal to 0: 25


#  SMS spam data
The file sms.csv contains a selection of SMS messages which have been classified as either 'spam' or 'ham'. These data have been adapted from the UCI Machine Learning Repository. There are a total of 5574 SMS, of which 747 have been labelled as spam.

Notes on CSV format:

- no header record and
- fields are separated by a semicolon (this is not the default separator).

Data dictionary:
- id — record identifier
- text — content of SMS message
- label — spam or ham (integer; 0 = ham and 1 = spam)

In [35]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Specify column names and types
schema = StructType([
    StructField("id", IntegerType()),
    StructField("text", StringType()),
    StructField("label", IntegerType())
])

# Load data from a delimited file
sms = spark.read.csv("sms.csv", sep=';', header=False, schema=schema)

# View the first five records
sms.show(5)


# Print schema of DataFrame
sms.printSchema()

+---+--------------------+-----+
| id|                text|label|
+---+--------------------+-----+
|  1|Sorry, I'll call ...|    0|
|  2|Dont worry. I gue...|    0|
|  3|Call FREEPHONE 08...|    1|
|  4|Win a 1000 cash p...|    1|
|  5|Go until jurong p...|    0|
+---+--------------------+-----+
only showing top 5 rows

root
 |-- id: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- label: integer (nullable = true)



##  Data Preprocessing:
- Removing puctuation and numbers: by using of regular exression or REGEX
- Tokenizing
- StopWordsRemover
- Feature hashing: After these two steps the document might contain a larg variety of words. It is better to convert it to numbers.  numFeatures=1024: The largest number which can be produced by hashing trick. It  should be big to capture diversity of the words
- Convert to TF-IDF representation. The TF-IDF matrix reflects how important a word is to each document. It takes into account both the frequency of the word within each document but also the frequency of the word across all of the documents in the collection.

- Dealing with commond words appear frequently across many documents.  
If a word appears in many documents then it is less usefull for buidling a classifier. we weight the number of counts for a word in a particular document against how frequenty that word occurs across all documents. To do this we reduce the effective count for more common  words giving what is known as the 'Inverse document frequency.'

In [36]:
# Import the necessary functions
from pyspark.sql.functions import regexp_replace
from pyspark.ml.feature import Tokenizer

# Remove punctuation (REGEX provided) and numbers
wrangled = sms.withColumn('text', regexp_replace(sms.text, '[_():;,.!?\\-]', ' '))
wrangled = wrangled.withColumn('text', regexp_replace(wrangled.text, '[0-9]', ' '))

# Merge multiple spaces
wrangled = wrangled.withColumn('text', regexp_replace(wrangled.text, ' +', ' '))

# Split the text into words
wrangled = Tokenizer(inputCol='text', outputCol='words').transform(wrangled)

wrangled.show(4, truncate=False)

+---+----------------------------------+-----+------------------------------------------+
|id |text                              |label|words                                     |
+---+----------------------------------+-----+------------------------------------------+
|1  |Sorry I'll call later in meeting  |0    |[sorry, i'll, call, later, in, meeting]   |
|2  |Dont worry I guess he's busy      |0    |[dont, worry, i, guess, he's, busy]       |
|3  |Call FREEPHONE now                |1    |[call, freephone, now]                    |
|4  |Win a cash prize or a prize worth |1    |[win, a, cash, prize, or, a, prize, worth]|
+---+----------------------------------+-----+------------------------------------------+
only showing top 4 rows



In [37]:
from pyspark.ml.feature import StopWordsRemover, HashingTF, IDF

# Remove stop words.
wrangled = StopWordsRemover(inputCol='words', outputCol='terms')\
      .transform(wrangled)

# Apply the hashing trick
wrangled = HashingTF(inputCol='terms', outputCol='hash', numFeatures=1024)\
      .transform(wrangled)

# Convert hashed symbols to TF-IDF
tf_idf = IDF(inputCol='hash', outputCol='features')\
      .fit(wrangled).transform(wrangled)
      
tf_idf.select('terms', 'features').show(4, truncate=False)

+--------------------------------+----------------------------------------------------------------------------------------------------+
|terms                           |features                                                                                            |
+--------------------------------+----------------------------------------------------------------------------------------------------+
|[sorry, call, later, meeting]   |(1024,[138,344,378,1006],[2.2391682769656747,2.892706319430574,3.684405173719015,4.244020961654438])|
|[dont, worry, guess, busy]      |(1024,[53,233,329,858],[4.618714411095849,3.557143394108088,4.618714411095849,4.937168142214383])   |
|[call, freephone]               |(1024,[138,396],[2.2391682769656747,3.3843005812686773])                                            |
|[win, cash, prize, prize, worth]|(1024,[31,69,387,428],[3.7897656893768414,7.284881949239966,4.4671645129686475,3.898659777615979])  |
+--------------------------------+--------------

 ## Logistic Regression

In [39]:
# Split the data into training and testing sets
sms_train, sms_test = tf_idf.randomSplit([0.5, 0.5], seed=13)

# Fit a Logistic Regression model to the training data
logistic = LogisticRegression(regParam=0.2).fit(sms_train)

# Make predictions on the testing data
prediction = logistic.transform(sms_test)

# Create a confusion matrix, comparing predictions to known labels
prediction.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|  119|
|    0|       0.0| 2431|
|    1|       1.0|  272|
|    0|       1.0|   11|
+-----+----------+-----+

